In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

output_csv = "programs_info.csv"
# ชื่อ field ที่ต้องการเท่านั้น (เปรียบเทียบแบบตรงตัว)
target_fields = ['วิศวกรรมศาสตร์ วิศวกรรมคอมพิวเตอร์', 'วิศวกรรมศาสตร์ วิศวกรรมปัญญาประดิษฐ์']
keywords = target_fields  # ถ้าค้นตามชื่อ field

browser = webdriver.Chrome()

with open(output_csv, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["university", "faculty", "field", "program_name", "program_type", "cost"])

    for keyword in keywords:
        print(f'\n🔍 กำลังค้นหา: {keyword}')
        browser.get('https://mytcas.com/')
        time.sleep(5)

        # รอ input ช่อง search (เปลี่ยน selector ตามหน้าเว็บจริง)
        search_box = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.ID, 'search'))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        time.sleep(3)
        search_box.send_keys(Keys.RETURN)

        try:
            WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 't-programs'))
            )
        except:
            print("⛔ ไม่พบ ul.t-programs")
            continue

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        ul = soup.find('ul', class_='t-programs')
        links = []

        if ul:
            for li in ul.find_all('li'):
                a = li.find('a', href=True)
                if a:
                    links.append(a['href'])

        print(f"🔗 พบลิงก์ทั้งหมด: {len(links)} ลิงก์")

        for i, link in enumerate(links, 1):
            print(f"  ({i}) เปิดลิงก์: {link}")
            try:
                browser.get(link)
                WebDriverWait(browser, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'dt'))
                )
                detail_soup = BeautifulSoup(browser.page_source, 'html.parser')

                # 1. ชื่อมหาวิทยาลัย
                university = "-"
                uni_tag = detail_soup.select_one('span.name a')
                if uni_tag:
                    university = uni_tag.get_text(strip=True)

                # 2. ชื่อคณะ
                faculty = "-"
                fac_tag = detail_soup.select_one('small a')
                if fac_tag:
                    faculty = fac_tag.get_text(strip=True)

                # 3. ชื่อหลักสูตร (ภาษาอังกฤษ)
                program_name = "-"
                for dt in detail_soup.find_all('dt'):
                    if "ชื่อหลักสูตรภาษาอังกฤษ" in dt.get_text(strip=True):
                        dd = dt.find_next_sibling('dd')
                        if dd:
                            program_name = dd.get_text(strip=True)
                        break

                # 4. ประเภทหลักสูตร
                program_type = "-"
                for dt in detail_soup.find_all('dt'):
                    if "ประเภทหลักสูตร" in dt.get_text(strip=True):
                        dd = dt.find_next_sibling('dd')
                        if dd:
                            program_type = dd.get_text(strip=True)
                        break

                # 5. ค่าใช้จ่าย
                cost = "-"
                for dt in detail_soup.find_all('dt'):
                    if dt.get_text(strip=True) == "ค่าใช้จ่าย":
                        dd = dt.find_next_sibling('dd')
                        if dd:
                            cost = dd.get_text(strip=True)
                        break

                writer.writerow([university, faculty, program_name, program_type, cost])

            except Exception as e:
                print(f"    ❗ Error: {e}")

print("\n✅ เสร็จสิ้น ดึงข้อมูลและบันทึก CSV เฉพาะคณะที่ต้องการเรียบร้อย")
browser.quit()


🔍 กำลังค้นหา: วิศวกรรมศาสตร์ วิศวกรรมคอมพิวเตอร์
🔗 พบลิงก์ทั้งหมด: 44 ลิงก์
  (1) เปิดลิงก์: https://course.mytcas.com/programs/10010121300501A?
  (2) เปิดลิงก์: https://course.mytcas.com/programs/10010121300599A?
  (3) เปิดลิงก์: https://course.mytcas.com/programs/10020105300501A?
  (4) เปิดลิงก์: https://course.mytcas.com/programs/10020220300501A?
  (5) เปิดลิงก์: https://course.mytcas.com/programs/10020328300502A?
  (6) เปิดลิงก์: https://course.mytcas.com/programs/10020430300501A?
  (7) เปิดลิงก์: https://course.mytcas.com/programs/10030104300501A?
  (8) เปิดลิงก์: https://course.mytcas.com/programs/10040106300501A?
  (9) เปิดลิงก์: https://course.mytcas.com/programs/10040106300502E?
  (10) เปิดลิงก์: https://course.mytcas.com/programs/10050210300501A?
  (11) เปิดลิงก์: https://course.mytcas.com/programs/10060112300501A?
  (12) เปิดลิงก์: https://course.mytcas.com/programs/10090209300501A?
  (13) เปิดลิงก์: https://course.mytcas.com/programs/10100101300501A?
  (14) เปิดลิงก์: http